In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize

/Users/melihtaki/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
def getData(path):
    url = "https://api-nprd.traxes.io/hackathon"
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"
    headers = {
            "x-api-key": api_key  # Include the API key in the header
        }
    response = requests.get(url+path, headers=headers)
    
    ## Check response
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Equipment Data

In [4]:
json_data = getData("/equipment/GetAllTransformers")
flat_data = json_normalize(json_data, sep='_')  # Flatten the nested JSON


# Convert the JSON data to a Pandas DataFrame
at_df = pd.DataFrame(flat_data)
print(at_df.shape)
at_df.head(2)


(17, 16)


,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8
1,50282127,40.0,11,ONAF1,2015,2015-08-07T00:00:00,no wall,15.51,149.18,22.7,46.8,62.8,0.72,1.32,1.16,13.8


In [32]:
equipment_ids = list(at_df.equipmentId)

all_load_data = []
for equipment in equipment_ids:
    json_data = getData(f"/equipment/GetTransformerLoad?equipmentId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:45:00")
    equipment_load_df = pd.DataFrame(json_data)
    all_load_data.append(equipment_load_df)

load_df = pd.concat(all_load_data)
print(load_df.shape)
load_df.head(2)

(1093508, 3)


,equipmentId,dateTime,load
0,50033827,2023-04-30T22:00:00,12294.37
1,50033827,2023-04-30T22:15:00,12115.62


In [33]:
equipment_ids = list(at_df.equipmentId)

all_temp_data = []
for equipment in equipment_ids:
    json_data = getData(f"/equipment/GetTransformerTemperature?equipmentId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:45:00")
    equipment_temp_df = pd.DataFrame(json_data)
    all_temp_data.append(equipment_temp_df)

temp_df = pd.concat(all_temp_data)
print(temp_df.shape)
temp_df.head(2)

(997220, 3)


,equipmentId,dateTime,hotspotTemperature
0,50033827,2023-04-30T22:00:00,41.02
1,50033827,2023-04-30T22:15:00,41.38


In [34]:
transformer_data = pd.merge(load_df, temp_df, on=('equipmentId','dateTime'), how='left')
transformer_data.shape
transformer_data.to_csv("../data/transformer_data.csv", index=False)

In [6]:
at_df.to_csv("../data/transformers_info_data.csv", index=False)

# Weather Data

In [4]:
at_df = pd.read_csv("../data/transformers_info_data.csv")

In [14]:
len(location_ids)

17

In [16]:
location_ids = list(at_df.locationId.unique())
len(location_ids)

11

In [20]:
location_ids = list(at_df.locationId.unique())

all_weather_temp_data = []
for locId in location_ids:
    json_data = getData(f"/weather/GetOutsideTemperature?locationId={locId}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00")
    equipment_temp_df = pd.DataFrame(json_data)
    all_weather_temp_data.append(equipment_temp_df)

weather_temp_df = pd.concat(all_weather_temp_data)
print(weather_temp_df.shape)
weather_temp_df.head(2)

(176891, 3)


,locationId,dateTime,temperature
0,26,2023-04-30T22:00:00,12.77
1,26,2023-04-30T23:00:00,12.26


In [21]:
location_ids = list(at_df.locationId.unique())

all_weather_temp_data = []
for locId in location_ids:
    json_data = getData(f"/weather/GetWind?locationId={locId}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00")
    equipment_temp_df = pd.DataFrame(json_data)
    all_weather_temp_data.append(equipment_temp_df)

weather_wind_df = pd.concat(all_weather_temp_data)
print(weather_wind_df.shape)
weather_wind_df.head(2)

(176891, 3)


,locationId,dateTime,windSpeed
0,26,2023-04-30T22:00:00,2.24
1,26,2023-04-30T23:00:00,2.05


In [22]:
location_ids = list(at_df.locationId.unique())

all_weather_temp_data = []
for locId in location_ids:
    json_data = getData(f"/weather/GetHumidity?locationId={locId}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00")
    equipment_temp_df = pd.DataFrame(json_data)
    all_weather_temp_data.append(equipment_temp_df)

weather_humi_df = pd.concat(all_weather_temp_data)
print(weather_humi_df.shape)
weather_humi_df.head(2)

(176891, 3)


,locationId,dateTime,relativeHumidityPercentage
0,26,2023-04-30T22:00:00,68.40
1,26,2023-04-30T23:00:00,70.96


In [30]:
# Merge all weather data (temperature, wind speed, humidity) on locationId and dateTime
weather_df = pd.merge(weather_temp_df, weather_wind_df, on=['locationId', 'dateTime'], how='inner')
weather_df = pd.merge(weather_df, weather_humi_df, on=['locationId', 'dateTime'], how='inner')

# Verify the merged dataframe
print(weather_df.shape)
weather_df.head()


(176891, 5)


,locationId,dateTime,temperature,windSpeed,relativeHumidityPercentage
0,26,2023-04-30T22:00:00,12.77,2.24,68.40
1,26,2023-04-30T23:00:00,12.26,2.05,70.96
2,26,2023-05-01T00:00:00,11.73,1.60,74.35
3,26,2023-05-01T01:00:00,11.35,1.39,77.63
4,26,2023-05-01T02:00:00,10.80,1.13,80.86


In [31]:
# Afficher les données météo pour locationId = 2
print(weather_df[weather_df['locationId'] == 2].head())


        locationId             dateTime  temperature  windSpeed  \
112567           2  2023-04-30T22:00:00        13.92       2.43   
112568           2  2023-04-30T23:00:00        13.66       2.29   
112569           2  2023-05-01T00:00:00        12.92       1.69   
112570           2  2023-05-01T01:00:00        12.51       1.14   
112571           2  2023-05-01T02:00:00        11.80       1.01   

        relativeHumidityPercentage  
112567                       62.84  
112568                       62.86  
112569                       66.46  
112570                       68.63  
112571                       73.98  


In [27]:
# Resample weather data to 15-minute intervals using linear interpolation
weather_df['dateTime'] = pd.to_datetime(weather_df['dateTime'])

# Drop any duplicate rows based on dateTime and locationId to ensure unique index
weather_df = weather_df.drop_duplicates(subset=['dateTime', 'locationId'])

# Set multi-index using both dateTime and locationId
weather_df = weather_df.set_index(['dateTime', 'locationId'])

# Sort index to ensure proper interpolation 
weather_df = weather_df.sort_index()

# Create new date range with 15 min frequency for each locationId
locations = weather_df.index.get_level_values('locationId').unique()
resampled_dfs = []

for loc in locations:
    # Get data for this location
    loc_df = weather_df.xs(loc, level='locationId')
    
    # Resample and interpolate
    loc_df = loc_df.resample('15T').interpolate(method='linear')
    
    # Add location back as index
    loc_df['locationId'] = loc
    loc_df = loc_df.set_index('locationId', append=True)
    
    resampled_dfs.append(loc_df)

# Combine all resampled dataframes
weather_df = pd.concat(resampled_dfs)

# Reset index to make dateTime and locationId columns again
weather_df = weather_df.reset_index()

print(weather_df.shape)
weather_df.head()


(707531, 5)


/var/folders/n9/q8bvx5ln4m10k1kwfc2w0zd40000gn/T/ipykernel_11378/3737991561.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  loc_df = loc_df.resample('15T').interpolate(method='linear')
/var/folders/n9/q8bvx5ln4m10k1kwfc2w0zd40000gn/T/ipykernel_11378/3737991561.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  loc_df = loc_df.resample('15T').interpolate(method='linear')
/var/folders/n9/q8bvx5ln4m10k1kwfc2w0zd40000gn/T/ipykernel_11378/3737991561.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  loc_df = loc_df.resample('15T').interpolate(method='linear')
/var/folders/n9/q8bvx5ln4m10k1kwfc2w0zd40000gn/T/ipykernel_11378/3737991561.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  loc_df = loc_df.resample('15T').interpolate(method='linear')
/var/folders/n9/

,dateTime,locationId,temperature,windSpeed,relativeHumidityPercentage
0,2023-04-30 22:00:00,2,13.920,2.430,62.840
1,2023-04-30 22:15:00,2,13.855,2.395,62.845
2,2023-04-30 22:30:00,2,13.790,2.360,62.850
3,2023-04-30 22:45:00,2,13.725,2.325,62.855
4,2023-04-30 23:00:00,2,13.660,2.290,62.860


In [28]:
# Sauvegarder le DataFrame weather_df au format CSV
weather_df.to_csv('weather_data.csv', index=False)
print("Les données météo ont été sauvegardées dans 'weather_data.csv'")


Les données météo ont été sauvegardées dans 'weather_data.csv'
